<a href="https://colab.research.google.com/github/valogonor/ALS-Data-Engineer-Exercise/blob/master/ALS_Data_Engineer_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
cons = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv')
cons.head()

,cons_id,prefix,firstname,middlename,lastname,suffix,salutation,gender,birth_dt,title,employer,occupation,income,source,subsource,userid,password,is_validated,is_banned,change_password_next_login,consent_type_id,create_dt,create_app,create_user,modified_dt,modified_app,modified_user,status,note
0,1,NaN,NaN,Lee,NaN,MD,NaN,E,NaN,vSkSIzEQJdXnqeTTTXSG,NaN,NaN,2.971606e+10,google,NaN,3663,_kXcXaoK7i,1,0,0,5958,"Fri, 1983-08-26 06:02:03",1484,6162,"Sun, 2015-12-27 09:28:02",4022,6349,1,NaN
1,2,NaN,NaN,NaN,NaN,II,boFqBKgLlSgEZsFrgCZd,E,"Mon, 2004-11-15",NaN,NaN,NaN,6.717468e+02,facebook,pRzBAZSGNScwCyreCEYr,7125,Ll3ZUxnh*9,0,0,1,4236,"Mon, 1979-03-05 21:08:54",4176,5476,"Tue, 1989-06-20 13:28:57",9010,5698,1,NaN
2,3,NaN,NaN,David,King,NaN,NaN,D,"Fri, 1994-04-08",bxGxufoNzpKvjwNIxgRj,iPUtgXtqIBEaxQxaMMsr,NaN,NaN,NaN,UAWXnALxxBXmwbPibFdw,5202,&@sU8IaE+L,1,0,1,1263,"Fri, 2008-08-22 19:20:28",4702,8239,"Fri, 2020-06-05 18:13:57",8837,1175,1,NaN
3,4,Ms.,Mark,Gregg,NaN,NaN,NaN,J,NaN,NaN,pFxPMIlwpLmWhylhVPlq,NaN,8.233324e+09,google,NaN,1591,5vVPwqxK)g,0,0,0,949,"Sun, 1984-04-29 11:18:18",7096,7875,"Tue, 2012-01-31 07:26:35",7529,3984,1,NaN
4,5,NaN,NaN,Logan,NaN,DDS,ArvPJuEozHPaqbirovMi,D,"Sun, 2008-08-10",kXKlKCyCBoXUBVejPOVO,NaN,QpESbZrLQlRIjKabyAKb,NaN,NaN,NaN,4578,S%3cXL#sKt,1,0,0,7126,"Thu, 2007-07-19 18:28:09",706,5727,"Tue, 1971-04-27 06:53:53",9356,6062,1,NaN


In [2]:
email = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv')
email.head()

,cons_email_id,cons_id,cons_email_type_id,is_primary,email,canonical_local_part,domain,double_validation,create_dt,create_app,create_user,modified_dt,modified_app,modified_user,status,note
0,1,548198,3361,1,xmartinez@vincent.com,NaN,gmail.com,NaN,"Wed, 1994-01-26 23:49:16",4072,9954,"Sat, 2014-04-19 19:10:39",1990,7595,1,NaN
1,2,491137,2474,1,hmiller@haynes.biz,jqCyozTDojYuylQPTHfm,hotmail.com,NaN,"Thu, 1999-12-09 06:18:27",1600,5716,"Sat, 1984-07-14 05:55:27",4686,3248,1,NaN
2,3,413429,5175,1,aaron64@yahoo.com,FCBeBiVoqnnKDWjnllhN,yahoo.com,kRLZexQEwYIMbwFNvQxg,"Wed, 1992-11-18 16:46:27",7358,2685,"Sun, 1995-12-24 13:13:01",3857,7405,1,NaN
3,4,347346,4117,1,wyattvincent@hotmail.com,NaN,gmail.com,zSbfmlqXimGyWVBUGdQg,"Sat, 1983-11-26 16:49:14",881,3444,"Sun, 1975-01-19 14:32:56",8713,7713,1,NaN
4,5,443000,6781,1,tspencer@hotmail.com,VaQIYlKcUkIywkKKEptD,gmail.com,NaN,"Wed, 2000-11-15 13:28:34",5380,5456,"Sun, 1994-03-13 16:38:37",765,8618,1,NaN


In [3]:
subs = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv')
subs.head()

,cons_email_chapter_subscription_id,cons_email_id,chapter_id,isunsub,unsub_dt,modified_dt
0,1,332188,1,1,"Sat, 1971-06-12 15:38:44","Thu, 1990-06-28 10:54:20"
1,2,536526,1,1,"Wed, 2006-07-12 01:50:45","Thu, 1979-09-20 06:02:35"
2,3,134711,1,1,"Tue, 1987-01-06 13:05:15","Sun, 1974-03-03 15:11:50"
3,4,660345,1,1,"Sat, 2016-08-06 11:06:09","Wed, 1995-09-13 23:45:03"
4,5,184268,1,1,"Sun, 2000-05-28 02:20:45","Sat, 1983-12-10 08:09:58"


In [4]:
len(cons), len(email), len(subs)

(700000, 1400000, 350000)

In [5]:
people = cons[['cons_id']]
people.head()

,cons_id
0,1
1,2
2,3
3,4
4,5


In [6]:
# I chose an outer join in order to keep all the available information.
people = people.merge(email[['cons_email_id', 'cons_id', 'email']], how='outer', on='cons_id')
people.head()

,cons_id,cons_email_id,email
0,1,546912.0,daniel72@hudson.com
1,1,560549.0,misty66@walsh-hampton.biz
2,1,1244042.0,lucassmith@yahoo.com
3,2,415587.0,caustin@spears-carson.com
4,2,642652.0,deborah57@shaffer-reed.org


In [7]:
len(people)

1494361

In [8]:
people = people.merge(cons[['cons_id', 'source']], on='cons_id', how='outer')
people.head()

,cons_id,cons_email_id,email,source
0,1,546912.0,daniel72@hudson.com,google
1,1,560549.0,misty66@walsh-hampton.biz,google
2,1,1244042.0,lucassmith@yahoo.com,google
3,2,415587.0,caustin@spears-carson.com,facebook
4,2,642652.0,deborah57@shaffer-reed.org,facebook


In [9]:
people = people.merge(subs[['cons_email_id', 'isunsub']], on='cons_email_id', how='outer')
people.head()

,cons_id,cons_email_id,email,source,isunsub
0,1,546912.0,daniel72@hudson.com,google,NaN
1,1,560549.0,misty66@walsh-hampton.biz,google,NaN
2,1,1244042.0,lucassmith@yahoo.com,google,NaN
3,2,415587.0,caustin@spears-carson.com,facebook,1.0
4,2,642652.0,deborah57@shaffer-reed.org,facebook,1.0


In [10]:
len(people)

1568877

In [11]:
people = people.merge(cons[['cons_id', 'create_dt', 'modified_dt']], on='cons_id', how='outer')
people.head()

,cons_id,cons_email_id,email,source,isunsub,create_dt,modified_dt
0,1,546912.0,daniel72@hudson.com,google,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02"
1,1,560549.0,misty66@walsh-hampton.biz,google,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02"
2,1,1244042.0,lucassmith@yahoo.com,google,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02"
3,2,415587.0,caustin@spears-carson.com,facebook,1.0,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57"
4,2,642652.0,deborah57@shaffer-reed.org,facebook,1.0,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57"


In [12]:
people = people.rename(columns={'source': 'code','isunsub': 'is_unsub', 'create_dt': 'created_dt', 'modified_dt': 'updated_dt'})
people.head()

,cons_id,cons_email_id,email,code,is_unsub,created_dt,updated_dt
0,1,546912.0,daniel72@hudson.com,google,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02"
1,1,560549.0,misty66@walsh-hampton.biz,google,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02"
2,1,1244042.0,lucassmith@yahoo.com,google,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02"
3,2,415587.0,caustin@spears-carson.com,facebook,1.0,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57"
4,2,642652.0,deborah57@shaffer-reed.org,facebook,1.0,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57"


In [13]:
people.columns

Index(['cons_id', 'cons_email_id', 'email', 'code', 'is_unsub', 'created_dt',
       'updated_dt'],
      dtype='object')

In [14]:
for col in people.columns:
    print(col, type(people[col][0]))

cons_id <class 'numpy.int64'>
cons_email_id <class 'numpy.float64'>
email <class 'str'>
code <class 'str'>
is_unsub <class 'numpy.float64'>
created_dt <class 'str'>
updated_dt <class 'str'>


In [15]:
people.dtypes

cons_id            int64
cons_email_id    float64
email             object
code              object
is_unsub         float64
created_dt        object
updated_dt        object
dtype: object

In [16]:
people['is_unsub'].value_counts()

1.0    315474
0.0     34526
Name: is_unsub, dtype: int64

In [17]:
len(people)

1568877

In [18]:
%%time
# Converting dates to datetime
cols = ['created_dt', 'updated_dt']
for col in cols:
    people[col] = pd.to_datetime(people[col])

CPU times: user 5min 22s, sys: 22.8 ms, total: 5min 22s
Wall time: 5min 22s


In [19]:
people.isna().sum()

cons_id                0
cons_email_id      94361
email              94361
code              783783
is_unsub         1218877
created_dt             0
updated_dt             0
dtype: int64

In [20]:
# Converting ID and boolean back to ints
# I chose to leave NaNs or blanks in the dataframe so that the information can
# be filled in as it is acquired
cols = ['cons_email_id', 'is_unsub']
for col in cols:
    people[col] = people[col].apply(lambda x: int(x) if x == x else '')
people.head()

,cons_id,cons_email_id,email,code,is_unsub,created_dt,updated_dt
0,1,546912,daniel72@hudson.com,google,,1983-08-26 06:02:03,2015-12-27 09:28:02
1,1,560549,misty66@walsh-hampton.biz,google,,1983-08-26 06:02:03,2015-12-27 09:28:02
2,1,1244042,lucassmith@yahoo.com,google,,1983-08-26 06:02:03,2015-12-27 09:28:02
3,2,415587,caustin@spears-carson.com,facebook,1,1979-03-05 21:08:54,1989-06-20 13:28:57
4,2,642652,deborah57@shaffer-reed.org,facebook,1,1979-03-05 21:08:54,1989-06-20 13:28:57


In [21]:
people.dtypes

cons_id                   int64
cons_email_id            object
email                    object
code                     object
is_unsub                 object
created_dt       datetime64[ns]
updated_dt       datetime64[ns]
dtype: object

In [22]:
people.isna().sum()

cons_id               0
cons_email_id         0
email             94361
code             783783
is_unsub              0
created_dt            0
updated_dt            0
dtype: int64

In [23]:
people.tail()

,cons_id,cons_email_id,email,code,is_unsub,created_dt,updated_dt
1568872,699999,1121156,jennifer52@olsen.org,google,,1998-06-02 01:24:00,2014-06-01 11:16:45
1568873,700000,92309,christie54@gmail.com,NaN,,1989-02-15 17:34:48,2020-04-30 19:39:18
1568874,700000,568843,shawnguzman@hotmail.com,NaN,,1989-02-15 17:34:48,2020-04-30 19:39:18
1568875,700000,963789,hunterjohnny@gmail.com,NaN,,1989-02-15 17:34:48,2020-04-30 19:39:18
1568876,700000,1199889,lisa34@gray.com,NaN,,1989-02-15 17:34:48,2020-04-30 19:39:18


In [24]:
cons['create_dt'] = pd.to_datetime(cons['create_dt'])
cons['create_dt'] = cons['create_dt'].dt.date

In [25]:
acquisitions = cons[['cons_id', 'create_dt']]
acquisitions.head()

,cons_id,create_dt
0,1,1983-08-26
1,2,1979-03-05
2,3,2008-08-22
3,4,1984-04-29
4,5,2007-07-19


In [26]:
# Grouping the acquisitions by date
dates = acquisitions.groupby('create_dt').groups

In [27]:
len(dates)

18445

In [29]:
data = {'acquisition_date': list(dates.keys()), 'acquisitions': [len(dates[date]) for date in dates]}

In [30]:
acquisition_facts = pd.DataFrame(data=data)
acquisition_facts.head()

,acquisition_date,acquisitions
0,1970-01-01,34
1,1970-01-02,38
2,1970-01-03,32
3,1970-01-04,39
4,1970-01-05,44


In [31]:
# Making sure the number of dates in the dataframe match the number of dates in
# the dictionary
len(acquisition_facts) == len(dates)

True

In [32]:
people.to_csv('people.csv', index=False)

In [33]:
acquisition_facts.to_csv('acquisition_facts.csv', index=False)